In [2]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep

In [3]:
# Importing cross currency sheet
cryptos = pd.read_csv('crosspairs.csv')

'''
for idx, row in cryptos.iterrows():
    symbol = row['Symbol']
    exchanges = row.dropna().index.values[2:]
    
exchanges
'''

"\nfor idx, row in cryptos.iterrows():\n    symbol = row['Symbol']\n    exchanges = row.dropna().index.values[2:]\n    \nexchanges\n"

In [4]:
# Pull from Cryptopia
def pulltopia(symbol,base):
    # Fields:
    # AskPrice BaseVolume BidPrice BuyBaseVolume BuyVolume Change
    # Close High Label LastPrice Low Open SellBaseVolume 
    # SellVolume TradePairId Volume
    
    topiasite = 'https://www.cryptopia.co.nz/api/GetMarket/' + symbol + '_' + base + '/0.001'
    response = requests.get(topiasite)
    json_data = json.loads(response.text)
    return pd.DataFrame(json_data['Data'], index = [0])

# Pull from kucoin
def pullkucoin(symbol,base):
    kubuy = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-buy/'
    kusell = 'https://api.kucoin.com/v1/' + symbol + '-' + base + '/open/orders-sell/'
    respone = requests.get(kubuy)
    json_data_buy = json.loads(respone.text)
    response = requests.get(kusell)
    json_data_sell = json.loads(response.text)
    return pd.DataFrame({'BidPrice':[x[0] for x in json_data_buy['data']],
                         'BuyBaseVolume': [x[1] for x in json_data_buy['data']],
                         'AskPrice': [x[0] for x in json_data_sell['data']],
                         'SellBaseVolume': [x[1] for x in json_data_sell['data']]},
                        index = range(len(json_data_buy['data'])))

# Pull from Bittrex
def trexpull(symbol,base):
    trexsite = 'https://bittrex.com/api/v1.1/public/getorderbook?market=' + base + '-' + symbol + '&type=both'
    response = requests.get(trexsite)
    json_data = json.loads(response.text)
    return pd.concat([pd.DataFrame({'BidPrice': [x['Rate'] for x in json_data['result']['buy']],
                                    'BuyBaseVolume': [x['Quantity'] for x in json_data['result']['buy']]},
                                   index = range(len(json_data['result']['buy']))),
                      pd.DataFrame({'AskPrice': [x['Rate'] for x in json_data['result']['sell']],
                                    'SellBaseVolume': [x['Quantity'] for x in json_data['result']['sell']]},
                                   index = range(len(json_data['result']['sell'])))],
                     axis = 1)

# Pull from Quoine
def quoinepull(id):
    # currencypair id
    # UBTCETH-->75
    # UBTCBTC-->74
    # BTCUSD-->1
    # QASHBTC-->52
    # ETHBTC-->37
    # ETHUSD-->27
    # NEOUSD-->53
    # 
    quoinesite = 'https://api.quoine.com/products/' + str(id) + '/price_levels'
    response = requests.get(quoinesite)
    json_data = json.loads(response.text)
    return pd.DataFrame({'BidPrice': [x[0] for x in json_data['buy_price_levels']],
                         'BuyBaseVolume': [x[1] for x in json_data['buy_price_levels']],
                         'AskPrice': [x[0] for x in json_data['sell_price_levels']],
                         'SellBaseVolume': [x[1] for x in json_data['sell_price_levels']]},
                        index = range(len(json_data['buy_price_levels'])))

In [29]:
# Arbitrage from Kucoin and Trex

def kutrexarb(symbol,base):
    ku = pullkucoin(symbol,base)
    ku.columns = ['ku_' + colname for colname in ku]
    trex = trexpull(symbol,base)[:6]
    trex.columns = ['trex_' + colname for colname in trex]
    return pd.concat([ku,trex],axis = 1)

def checkarbs():
    pairs = pd.read_csv('kutrex.csv')
    currencypairs = []
    buyex = []
    sellex = []
    askprice = []
    bidprice = []
    arbamount = []
    for idx, row in pairs.iterrows():
        symbol = row['Symbol']
        base = row['Base']
        tempseries = kutrexarb(symbol,base).iloc[0]
        
        if tempseries['ku_AskPrice'] < tempseries['trex_BidPrice']:
            # Buy on Ku and sell on Trex
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Ku']
            sellex = sellex + ['Trex']
            askprice = askprice + [tempseries['ku_AskPrice']]
            bidprice = bidprice + [tempseries['trex_BidPrice']]
            arbamount = arbamount + [tempseries['trex_BidPrice']/tempseries['ku_AskPrice']-1]
        elif tempseries['trex_AskPrice'] < tempseries['ku_BidPrice']:
            # Buy on Trex ad sell on Ku
            currencypairs = currencypairs + [symbol + base]
            buyex = buyex + ['Trex']
            sellex = sellex + ['Ku']
            askprice = askprice + [tempseries['trex_AskPrice']]
            bidprice = bidprice + [tempseries['ku_BidPrice']]
            arbamount = arbamount + [tempseries['ku_BidPrice']/tempseries['trex_AskPrice']-1]
        else:
            continue
        sleep(0.1)
    
    df = pd.DataFrame({'BuyExchange': buyex,
                      'SellExchange': sellex,
                      'AskPrice': askprice,
                      'BidPrice': bidprice,
                      'ArbReturn': arbamount},
                     index = currencypairs)
    return df.sort_values(['ArbReturn'])

def ethbtcconvert():
    kutrex = kutrexarb('ETH','BTC')[['ku_AskPrice','ku_BidPrice',
                                    'trex_AskPrice','trex_BidPrice']]
    topia = pulltopia('ETH','BTC')[['AskPrice','BidPrice']]
    topia.columns = ['topia_AskPrice','topia_BidPrice']
    quoine = quoinepull(37)[['AskPrice','BidPrice']]
    quoine.columns = ['quoine_AskPrice','quoine_BidPrice']
    
    askdf = pd.concat([kutrex[['ku_AskPrice','trex_AskPrice']],
                      topia[['topia_AskPrice']],
                      quoine[['quoine_AskPrice']]],
                     axis = 1)
    
    bidf = pd.concat([kutrex[['ku_BidPrice','trex_BidPrice']],
                      topia[['topia_BidPrice']],
                      quoine[['quoine_BidPrice']]],
                     axis = 1)
    
    return askdf.iloc[0], bidf.iloc[0]

In [95]:
ask, bid = ethbtcconvert()

print(ask)
print(bid)

ku_AskPrice          0.100194
trex_AskPrice        0.099874
topia_AskPrice      0.0998352
quoine_AskPrice    0.09500000
Name: 0, dtype: object
ku_BidPrice          0.098624
trex_BidPrice       0.0997002
topia_BidPrice      0.0997452
quoine_BidPrice    0.09150001
Name: 0, dtype: object


In [152]:
def ethcomparison(exchangelst):
    site = 'https://coinmarketcap.com/currencies/ethereum/#markets'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]
    sources = []
    pairs = []
    volumes = []
    prices = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        elif (columns[2].split('target="_blank">')[1].split('</a></td>')[0] != 'ETH/BTC'):
            continue
        else:
            sources = sources + [columns[2].split('<td><a href="/exchanges/')[1].split('/"')[0]]
            pairs = pairs + [columns[2].split('target="_blank">')[1].split('</a></td>')[0]]
            volumes = volumes + [columns[4].split('data-btc="')[1].split('"')[0]]
            prices = prices + [columns[9].split('data-btc="')[1].split('"')[0]]

    ethbtcdf = pd.DataFrame({'Exchange': sources,
                            'Pair': pairs,
                            'BTCVolume': volumes,
                            'BTCPrice': prices},
                           index = sources)

    ethbtcdf['BTCPrice'] = pd.to_numeric(ethbtcdf['BTCPrice'])
    ethbtcdf['BTCVolume'] = pd.to_numeric(ethbtcdf['BTCVolume'])
    ethbtcdf = ethbtcdf.sort_values(['BTCPrice'], ascending = False)
    ethbtcdf = ethbtcdf[ethbtcdf['Exchange'].isin(exchangelst)]
    
    ethbtcdf['ArbReturn'] = ethbtcdf['BTCPrice']/ethbtcdf['BTCPrice'].min() - 1
    return ethbtcdf


In [154]:
filters = ['cryptopia','binance','bittrex','kucoin','quoine']
ethcomparison(filters)

,BTCPrice,BTCVolume,Exchange,Pair,ArbReturn
binance,0.099121,24450.800,binance,ETH/BTC,0.043412
cryptopia,0.099100,497.902,cryptopia,ETH/BTC,0.043191
bittrex,0.099090,7048.130,bittrex,ETH/BTC,0.043087
kucoin,0.098860,784.010,kucoin,ETH/BTC,0.040663
quoine,0.094997,515.643,quoine,ETH/BTC,0.000000
